In [ ]:
!pip install emoji transformers
!pip install sentencepiece
!pip install tensorflow_addons

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
from tqdm.notebook import tqdm
import emoji

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, Average
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_addons as tfa

from collections import Counter
from functools import partial

import transformers
from tokenizers import BertWordPieceTokenizer

AUTO = tf.data.experimental.AUTOTUNE

     |████████████████████████████████| 133kB 6.7MB/s 
     |████████████████████████████████| 2.0MB 23.0MB/s 
     |████████████████████████████████| 890kB 40.2MB/s 
     |████████████████████████████████| 3.2MB 25.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=5005ac76930ddf5d833ae9d8acb45db195be6f705d088e5145babbb78b16bfc4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 5.0MB/s 
     |████████████████████████████████| 706kB 5.0MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.28.231.106:8470


INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.28.231.106:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.28.231.106:8470


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


## Load Preprocessed Training Data From Kaggle

In [ ]:
data_folder = "/content/drive/MyDrive/CZ4034 - Information Retrieval/Data"

In [ ]:
df_sub = pd.read_csv(f'{data_folder}/preprocessed_kaggle_data.csv')
df_sub.head()

,id,comment_text,neutral,toxic_class
0,0000997932d777bf,explanation why the edits made under my userna...,0,0
1,000103f0d9cfb60f,d aww he matches this background colour i m se...,1,0
2,000113f07ec002fd,hey man i m really not trying to edit war it s...,0,0
3,0001b41b1c6bb37e,more i can t make any real suggestions on impr...,0,0
4,0001d958c54c6e35,you sir are my hero any chance remember what p...,1,0


In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Split Kaggle Dataset to Training set as well as test set to be used to test model after training
X_train, X_test, y_train, y_test = train_test_split(df_sub.comment_text.values, df_sub.id.values, test_size=0.10, random_state=42, stratify=df_sub[['toxic_class','neutral']])

In [ ]:
y_train.size

143613

In [ ]:
data_df = df_sub[df_sub['id'].isin(y_train)]
data_df.head()

,id,comment_text,neutral,toxic_class
0,0000997932d777bf,explanation why the edits made under my userna...,0,0
1,000103f0d9cfb60f,d aww he matches this background colour i m se...,1,0
2,000113f07ec002fd,hey man i m really not trying to edit war it s...,0,0
4,0001d958c54c6e35,you sir are my hero any chance remember what p...,1,0
5,00025465d4725e87,congratulations from me as well use the tools ...,1,0


In [ ]:
test_df = df_sub[df_sub['id'].isin(y_test)]
test_df.head()

,id,comment_text,neutral,toxic_class
3,0001b41b1c6bb37e,more i can t make any real suggestions on impr...,0,0
14,00070ef96486d6f9,oh and the girl above started her arguments wi...,1,0
38,00169857adbc989b,hi explicit can you block o fenian for edit wa...,1,0
41,00173958f46763a2,tfd i think we just eced responded to each oth...,0,0
83,00349c6325526c11,april thank you for experimenting with the pag...,0,0


In [ ]:
data_df.rename({'neutral':'subjectivity','toxic_class':'toxicity'},axis=1,inplace=True)
data_df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,id,comment_text,subjectivity,toxicity
0,0000997932d777bf,explanation why the edits made under my userna...,0,0
1,000103f0d9cfb60f,d aww he matches this background colour i m se...,1,0
2,000113f07ec002fd,hey man i m really not trying to edit war it s...,0,0
4,0001d958c54c6e35,you sir are my hero any chance remember what p...,1,0
5,00025465d4725e87,congratulations from me as well use the tools ...,1,0


In [ ]:
# tweets = pd.read_csv(f'{data_folder}/preprocessed_neg_train.csv')
# tweets.drop('id', axis=1, inplace=True)
# tweets['comment_text'] = tweets['comment_text'].astype(str)
# tweets.rename({'toxic':'toxicity'},axis=1,inplace=True)
# tweets.head()

In [ ]:
# def delete_repeated_char(text):
#     text = re.sub(r'(\w)\1{2,}', r'\1', text)
#     return text
# def remove_punc(tweet):
#     tweet =  tweet.translate(str.maketrans('', '', string.punctuation))
#     tweet = ' '.join([word for word in tweet.split()])
#     tweet = tweet.lower()
#     return tweet

In [ ]:
data_df.drop('id', axis=1, inplace=True)
data_df['comment_text'] = data_df['comment_text'].astype(str)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143613 entries, 0 to 159570
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   comment_text  143613 non-null  object
 1   subjectivity  143613 non-null  int64 
 2   toxicity      143613 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.4+ MB


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# data_df = data_df.append(tweets)
# data_df['comment_text'] = data_df['comment_text'].apply(delete_repeated_char).apply(remove_punc)
# data_df['count'] = data_df['comment_text'].str.split().map(len)
# drop_indexes = data_df.loc[data_df['count']==0].index.tolist()
# data_df = data_df.drop(drop_indexes)

# data_df = data_df.drop_duplicates(subset=['comment_text'])

# data_df = data_df.sample(frac=1, random_state=42)
# data_df.head()

In [ ]:
test_df.rename({'neutral':'subjectivity','toxic_class':'toxicity'},axis=1,inplace=True)
test_df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,id,comment_text,subjectivity,toxicity
3,0001b41b1c6bb37e,more i can t make any real suggestions on impr...,0,0
14,00070ef96486d6f9,oh and the girl above started her arguments wi...,1,0
38,00169857adbc989b,hi explicit can you block o fenian for edit wa...,1,0
41,00173958f46763a2,tfd i think we just eced responded to each oth...,0,0
83,00349c6325526c11,april thank you for experimenting with the pag...,0,0


In [ ]:
y_true = test_df['toxicity'].values

weight = Counter(y_true)

In [ ]:
print("Class weights: " + str(weight))

Class weights: Counter({0: 14429, 1: 1370, 2: 159})


## Functions For Processing TF Data

In [ ]:
MAX_LEN = 192
EPOCHS = 4
BATCH_SIZE = 16 * strategy.num_replicas_in_sync # for ensemble model
# BATCH_SIZE = 8 * strategy.num_replicas_in_sync  # for multihead model

In [ ]:
def one_hot(text,label=None):
  
  subjectivity_oh = tf.one_hot(label[0],2)
  toxicity_oh = tf.one_hot(label[1],3)

  return text, {'subjectivity': subjectivity_oh, 'toxicity': toxicity_oh}

In [ ]:
def one_hot_multi(text,label=None):
  
  subjectivity_oh = tf.one_hot(label[0],2)
  toxicity_oh = tf.one_hot(label[1],3)

  return text, {'subjectivity': subjectivity_oh, 'toxicity': toxicity_oh}

## Normal Ensemble Model Function

In [ ]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
             texts, 
             truncation=True, 
             return_token_type_ids=False,
             pad_to_max_length=True,
             max_length=maxlen)
    
    return np.array(enc_di['input_ids'])

def build_model(transformer, model_name, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    if model_name == 'bert-base-multilingual-uncased':
      sequence_output = transformer.bert(input_word_ids)[0]
    elif model_name == 'jplu/tf-xlm-roberta-base':
      sequence_output = transformer.roberta(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    # x = Dense(1024, activation='relu')(cls_token)
    # x = Dropout(0.5)(x)
    x1 = Dense(512, activation='relu')(cls_token)
    x1 = Dropout(0.2)(x1)
    x1 = Dense(256, activation='relu')(x1)
    x1 = Dropout(0.2)(x1)
    x1 = Dense(128, activation='relu')(x1)
    x1 = Dropout(0.2)(x1)
    x2 = Dense(512, activation='relu')(cls_token)
    x2 = Dropout(0.2)(x2)
    x2 = Dense(256, activation='relu')(x2)
    x2 = Dropout(0.2)(x2)
    x2 = Dense(128, activation='relu')(x2)
    x2 = Dropout(0.2)(x2)
    sub = Dense(2, activation='softmax', name='subjectivity')(x1)
    toxic = Dense(3, activation='softmax', name='toxicity')(x2)
    
    model = Model(inputs=input_word_ids, outputs=[sub,toxic])
    loss = 'categorical_crossentropy'
    model.compile(Adam(lr=1e-5), loss=[loss, loss],
    metrics={'subjectivity':'accuracy', 'toxicity':'accuracy'})
    
    return model

## Ensemble model tranining on Kaggle Data

In [ ]:
save_model_folder = "/content/drive/MyDrive/CZ4034 - Information Retrieval/models"

In [ ]:
MODEL_NAMES = ['bert-base-multilingual-uncased','jplu/tf-xlm-roberta-base']
models = [0 for i in range(len(MODEL_NAMES))]
test_probs = [0 for i in range(len(MODEL_NAMES))]
test_results = [0 for i in range(len(MODEL_NAMES))]

with strategy.scope():
    
    for i in range(len(MODEL_NAMES)):
        
        models[i] = transformers.TFAutoModel.from_pretrained(MODEL_NAMES[i])
        # models[i].trainable = False
        
        tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAMES[i])
        reviews = regular_encode(data_df.comment_text.astype(str).values.tolist(), tokenizer, maxlen=MAX_LEN)
    
        sss = StratifiedShuffleSplit(n_splits=4, random_state=42, test_size=0.2)
        X, y = reviews, data_df.subjectivity.astype(int).values
        z = data_df.toxicity.astype(int).values

        models[i] = build_model(models[i], MODEL_NAMES[i], max_len=MAX_LEN)
        models[i].summary()

        tf.keras.utils.plot_model(
          models[i],
          to_file=f"{save_model_folder}/{str(i)}_model.png",
          show_shapes=True,
          show_dtype=False,
          show_layer_names=True,
          rankdir="TB",
          expand_nested=False,
          dpi=300,
        )

        for train_index, valid_index in sss.split(X, y):

            X_train, X_valid = X[train_index], X[valid_index]
            y_train, y_valid = y[train_index], y[valid_index]
            z_train, z_valid = z[train_index], z[valid_index]

            train_df = pd.DataFrame()
            train_df['sub'] = y_train
            train_df['tox'] = z_train
            label_cols = train_df.columns[:]
            labels = train_df[label_cols].values

            y_true = train_df['tox'].values
            print(Counter(y_true))
            y_true = train_df['sub'].values
            print(Counter(y_true))

            train_dataset = (
                tf.data.Dataset
                .from_tensor_slices((X_train,labels))
                .map(one_hot, num_parallel_calls=AUTO)
                .repeat()
                .shuffle(2048)
                .batch(BATCH_SIZE)
                .prefetch(AUTO)
            )

            valid_df = pd.DataFrame()
            valid_df['sub'] = y_valid
            valid_df['tox'] = z_valid
            label_cols = valid_df.columns[:]
            labels = valid_df[label_cols].values

            valid_dataset = (
                tf.data.Dataset
                .from_tensor_slices((X_valid, labels))
                .map(one_hot,num_parallel_calls=AUTO)
                .batch(BATCH_SIZE)
                .cache()
                .prefetch(AUTO)
            )

            n_steps = X_train.shape[0] // BATCH_SIZE
            models[i].fit(
                train_dataset,
                steps_per_epoch=n_steps,
                validation_data=valid_dataset,
                epochs=EPOCHS
            )
            
        test_reviews = regular_encode(test_df.comment_text.astype(str).values.tolist(), tokenizer, maxlen=MAX_LEN)

        test_dataset = (
        tf.data.Dataset
        .from_tensor_slices((test_reviews))
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
        )

        test_probs[i] = models[i].predict(test_dataset, verbose=1)
        models[i].save(f"{save_model_folder}/ensemble_model_{str(i)}.h5")

Some layers from the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is d

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 192)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 167356416   input_word_ids[0][0]             
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 768)          0           bert[0][0]                       
__________________________________________________________________________________________________
dense (Dense)                   (None, 512)          393728      tf.__operators__.getitem[0][0]   
______________________________________________________________________________________________

897/897 [==============================] - 286s 235ms/step - loss: 0.7227 - subjectivity_loss: 0.4680 - toxicity_loss: 0.2547 - subjectivity_accuracy: 0.7763 - toxicity_accuracy: 0.9233 - val_loss: 0.4264 - val_subjectivity_loss: 0.2930 - val_toxicity_loss: 0.1334 - val_subjectivity_accuracy: 0.8780 - val_toxicity_accuracy: 0.9528
Epoch 2/4
897/897 [==============================] - 198s 221ms/step - loss: 0.4055 - subjectivity_loss: 0.2757 - toxicity_loss: 0.1297 - subjectivity_accuracy: 0.8822 - toxicity_accuracy: 0.9513 - val_loss: 0.3674 - val_subjectivity_loss: 0.2432 - val_toxicity_loss: 0.1242 - val_subjectivity_accuracy: 0.8970 - val_toxicity_accuracy: 0.9544
Epoch 3/4
897/897 [==============================] - 199s 222ms/step - loss: 0.3492 - subjectivity_loss: 0.2326 - toxicity_loss: 0.1165 - subjectivity_accuracy: 0.9025 - toxicity_accuracy: 0.9552 - val_loss: 0.3518 - val_subjectivity_loss: 0.2342 - val_toxicity_loss: 0.1176 - val_subjectivity_accuracy: 0.8982 - val_toxicit

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


125/125 [==============================] - 20s 126ms/step


...


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 192)]        0                                            
__________________________________________________________________________________________________
roberta (TFRobertaMainLayer)    TFBaseModelOutputWit 278043648   input_word_ids[0][0]             
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 768)          0           roberta[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 512)          393728      tf.__operators__.getitem_1[0][0] 
___________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:430: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 192001536 elements. This may consume a large amount of memory.
  num_elements)


897/897 [==============================] - 319s 267ms/step - loss: 1.0519 - subjectivity_loss: 0.6136 - toxicity_loss: 0.4384 - subjectivity_accuracy: 0.6727 - toxicity_accuracy: 0.8592 - val_loss: 0.6079 - val_subjectivity_loss: 0.4196 - val_toxicity_loss: 0.1883 - val_subjectivity_accuracy: 0.8091 - val_toxicity_accuracy: 0.9286
Epoch 2/4
897/897 [==============================] - 225s 251ms/step - loss: 0.5347 - subjectivity_loss: 0.3706 - toxicity_loss: 0.1641 - subjectivity_accuracy: 0.8367 - toxicity_accuracy: 0.9434 - val_loss: 0.4316 - val_subjectivity_loss: 0.3056 - val_toxicity_loss: 0.1260 - val_subjectivity_accuracy: 0.8659 - val_toxicity_accuracy: 0.9535
Epoch 3/4
897/897 [==============================] - 225s 250ms/step - loss: 0.4130 - subjectivity_loss: 0.2838 - toxicity_loss: 0.1292 - subjectivity_accuracy: 0.8792 - toxicity_accuracy: 0.9517 - val_loss: 0.3896 - val_subjectivity_loss: 0.2659 - val_toxicity_loss: 0.1237 - val_subjectivity_accuracy: 0.8893 - val_toxicit

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


125/125 [==============================] - 24s 137ms/step


## Ensemble Model Test Results On Kaggle Data

In [ ]:
test_probs[0][0]

array([[9.9983299e-01, 1.6702294e-04],
       [9.7398274e-03, 9.9026024e-01],
       [5.8313653e-02, 9.4168639e-01],
       ...,
       [9.9994791e-01, 5.2071457e-05],
       [9.9996579e-01, 3.4183930e-05],
       [3.7037894e-06, 9.9999630e-01]], dtype=float32)

In [ ]:
test_df['sub_pred'] = np.argmax(((test_probs[0][0] + test_probs[1][0])/2), axis = 1)
test_df['tox_pred'] = np.argmax(((test_probs[0][1] + test_probs[1][1])/2), axis = 1)
test_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,comment_text,subjectivity,toxicity,sub_pred,tox_pred
3,0001b41b1c6bb37e,more i can t make any real suggestions on impr...,0,0,0,0
14,00070ef96486d6f9,oh and the girl above started her arguments wi...,1,0,1,0
38,00169857adbc989b,hi explicit can you block o fenian for edit wa...,1,0,1,0
41,00173958f46763a2,tfd i think we just eced responded to each oth...,0,0,0,0
83,00349c6325526c11,april thank you for experimenting with the pag...,0,0,0,0


In [ ]:
from sklearn.metrics import f1_score,precision_score,recall_score

y_true = test_df['subjectivity'].values
y_pred = test_df['sub_pred'].values

print("subjectivity f1 score: " + str(f1_score(y_true, y_pred, average='micro')))
print("subjectivity precision score: " + str(precision_score(y_true, y_pred, average='micro')))
print("subjectivity recall score: " + str(recall_score(y_true, y_pred, average='micro')))
print('\n')
y_true = test_df['toxicity'].values
y_pred = test_df['tox_pred'].values
print("toxicity f1 score: " + str(f1_score(y_true, y_pred, average='micro')))
print("toxicity precision score: " + str(precision_score(y_true, y_pred, average='micro')))
print("toxicity recall score: " + str(recall_score(y_true, y_pred, average='micro')))

subjectivity f1 score: 0.9076325354054393
subjectivity precision score: 0.9076325354054393
subjectivity recall score: 0.9076325354054393


toxicity f1 score: 0.957638801854869
toxicity precision score: 0.957638801854869
toxicity recall score: 0.957638801854869


In [ ]:
from collections import Counter

y_true = test_df['sub_pred'].values

Counter(y_true)

Counter({0: 10758, 1: 5200})

In [ ]:
y_true = test_df['tox_pred'].values

Counter(y_true)

Counter({0: 14481, 1: 1318, 2: 159})

## Multi-head Model Function

In [ ]:
BATCH_SIZE = 8 * strategy.num_replicas_in_sync  # for multihead model
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
             texts, 
             truncation=True, 
             return_token_type_ids=False,
             pad_to_max_length=True,
             max_length=maxlen)
    
    return np.array(enc_di['input_ids'])

def build_model(max_len=512):
    
    transformer = transformers.TFAutoModel.from_pretrained('bert-base-multilingual-uncased')
    transformer2 = transformers.TFAutoModel.from_pretrained('jplu/tf-xlm-roberta-base')
    input_word_ids_1 = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids_1")
    input_word_ids_2 = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids_2")
    sequence_output1 = transformer.bert(input_word_ids_1)[0]
    sequence_output2 = transformer2.roberta(input_word_ids_2)[0]
    cls_token1 = sequence_output1[:, 0, :]
    cls_token2 = sequence_output2[:, 0, :]
    x = Average()([cls_token1,cls_token2])
    # x = Dropout(0.5)(x)
    x1 = Dense(512, activation='relu')(x)
    x1 = Dropout(0.2)(x1)
    x1 = Dense(256, activation='relu')(x1)
    x1 = Dropout(0.2)(x1)
    x1 = Dense(128, activation='relu')(x1)
    x1 = Dropout(0.2)(x1)
    x2 = Dense(512, activation='relu')(x)
    x2 = Dropout(0.2)(x2)
    x2 = Dense(256, activation='relu')(x2)
    x2 = Dropout(0.2)(x2)
    x2 = Dense(128, activation='relu')(x2)
    x2 = Dropout(0.2)(x2)
    sub = Dense(2, activation='softmax', name='subjectivity')(x1)
    toxic = Dense(3, activation='softmax', name='toxicity')(x2)
    
    model = Model(inputs=[input_word_ids_1,input_word_ids_2], outputs=[sub,toxic])
    loss = tfa.losses.SigmoidFocalCrossEntropy(reduction=tf.keras.losses.Reduction.AUTO)
    model.compile(Adam(lr=1e-6), loss=[loss, loss],
    metrics={'subjectivity':'accuracy', 'toxicity':'accuracy'})
    
    return model

## Multi-head Model Training on Kaggle Data

In [ ]:
MODEL_NAMES = ['bert-base-multilingual-uncased','jplu/tf-xlm-roberta-base']
models = [0 for i in range(len(MODEL_NAMES))]
test_probs = [0 for i in range(len(MODEL_NAMES))]
test_results = [0 for i in range(len(MODEL_NAMES))]

with strategy.scope():
        
    tokenizer1 = transformers.AutoTokenizer.from_pretrained(MODEL_NAMES[0])
    tokenizer2 = transformers.AutoTokenizer.from_pretrained(MODEL_NAMES[1])
    reviews1 = regular_encode(data_df.comment_text.astype(str).values.tolist(), tokenizer1, maxlen=MAX_LEN)
    reviews2 = regular_encode(data_df.comment_text.astype(str).values.tolist(), tokenizer2, maxlen=MAX_LEN)

    sss = StratifiedShuffleSplit(n_splits=4, random_state=42, test_size=0.2)
    X1, y1 = reviews1, data_df.subjectivity.astype(int).values
    z1 = data_df.toxicity.astype(int).values

    X2, y2 = reviews2, data_df.subjectivity.astype(int).values
    z2 = data_df.toxicity.astype(int).values

    model = build_model(max_len=MAX_LEN)
    model.summary()

    tf.keras.utils.plot_model(
      model,
      to_file=f"{save_model_folder}/multihead_model.png",
      show_shapes=True,
      show_dtype=False,
      show_layer_names=True,
      rankdir="TB",
      expand_nested=False,
      dpi=300,
    )

    for train_index, valid_index in sss.split(X1, y1):

        X_train1, X_valid1 = X1[train_index], X1[valid_index]
        y_train1, y_valid1 = y1[train_index], y1[valid_index]
        z_train1, z_valid1 = z1[train_index], z1[valid_index]

        X_train2, X_valid2 = X2[train_index], X2[valid_index]
        y_train2, y_valid2 = y2[train_index], y2[valid_index]
        z_train2, z_valid2 = z2[train_index], z2[valid_index]

        train_df = pd.DataFrame()
        train_df['sub'] = y_train1
        train_df['tox'] = z_train1
        label_cols = train_df.columns[:]
        labels = train_df[label_cols].values
        data = np.array([X_train1,X_train2]).reshape(-1,2,192)

        print(data.shape)

        y_true = train_df['tox'].values
        print(Counter(y_true))
        y_true = train_df['sub'].values
        print(Counter(y_true))

        train_dataset = (
            tf.data.Dataset
            .from_tensor_slices(({"input_word_ids_1":X_train1,"input_word_ids_2":X_train2},labels))
            .map(one_hot_multi, num_parallel_calls=AUTO)
            .repeat()
            .shuffle(2048)
            .batch(BATCH_SIZE)
            .prefetch(AUTO)
        )

        valid_df = pd.DataFrame()
        valid_df['sub'] = y_valid1
        valid_df['tox'] = z_valid1
        label_cols = valid_df.columns[:]
        labels = valid_df[label_cols].values
        data = np.array([X_valid1,X_valid2]).reshape(-1,2,192)


        valid_dataset = (
            tf.data.Dataset
            .from_tensor_slices(({"input_word_ids_1":X_valid1,"input_word_ids_2":X_valid2},labels))
            .map(one_hot_multi, num_parallel_calls=AUTO)
            .batch(BATCH_SIZE)
            .cache()
            .prefetch(AUTO)
        )

        n_steps = X_train.shape[0] // BATCH_SIZE
        model.fit(
            train_dataset,
            steps_per_epoch=n_steps,
            validation_data=valid_dataset,
            epochs=EPOCHS
        )
        
    test_reviews1 = regular_encode(test_df.comment_text.astype(str).values.tolist(), tokenizer1, maxlen=MAX_LEN)
    test_reviews2 = regular_encode(test_df.comment_text.astype(str).values.tolist(), tokenizer2, maxlen=MAX_LEN)

    test = np.array([test_reviews1,test_reviews2])

    test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(({"input_word_ids_1":test_reviews1,"input_word_ids_2":test_reviews2}))
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
    )

    test_probs = model.predict(test_dataset, verbose=1)
    model.save(f"{save_model_folder}/multihead_model.h5")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Some layers from the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Some layers from the model checkpoint at jplu/tf-xlm-roberta-base were not used when initializing TFXLMRobertaModel: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaModel were initialized from the model checkpoint at jplu/tf-xlm-roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids_1 (InputLayer)   [(None, 192)]        0                                            
__________________________________________________________________________________________________
input_word_ids_2 (InputLayer)   [(None, 192)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 167356416   input_word_ids_1[0][0]           
__________________________________________________________________________________________________
roberta (TFRobertaMainLayer)    TFBaseModelOutputWit 278043648   input_word_ids_2[0][0]           
______________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:430: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 192001536 elements. This may consume a large amount of memory.
  num_elements)


2243/2243 [==============================] - 848s 313ms/step - loss: 0.2381 - subjectivity_loss: 0.1514 - toxicity_loss: 0.0867 - subjectivity_accuracy: 0.6931 - toxicity_accuracy: 0.8491 - val_loss: 0.1430 - val_subjectivity_loss: 0.1043 - val_toxicity_loss: 0.0387 - val_subjectivity_accuracy: 0.8170 - val_toxicity_accuracy: 0.9382
Epoch 2/4
2243/2243 [==============================] - 672s 300ms/step - loss: 0.1492 - subjectivity_loss: 0.1074 - toxicity_loss: 0.0418 - subjectivity_accuracy: 0.8130 - toxicity_accuracy: 0.9376 - val_loss: 0.1239 - val_subjectivity_loss: 0.0900 - val_toxicity_loss: 0.0339 - val_subjectivity_accuracy: 0.8435 - val_toxicity_accuracy: 0.9426
Epoch 3/4
2243/2243 [==============================] - 674s 300ms/step - loss: 0.1285 - subjectivity_loss: 0.0933 - toxicity_loss: 0.0351 - subjectivity_accuracy: 0.8438 - toxicity_accuracy: 0.9443 - val_loss: 0.1122 - val_subjectivity_loss: 0.0802 - val_toxicity_loss: 0.0320 - val_subjectivity_accuracy: 0.8656 - val_t

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


250/250 [==============================] - 40s 131ms/step


## Multi-head Model Test Results On Kaggle Data

In [ ]:
test_probs[1]

array([[0.9883465 , 0.00640512, 0.00524835],
       [0.84542155, 0.09901538, 0.05556313],
       [0.81863934, 0.1192342 , 0.06212652],
       ...,
       [0.9829168 , 0.00935124, 0.00773196],
       [0.9639214 , 0.02126776, 0.01481093],
       [0.16320425, 0.38143745, 0.4553583 ]], dtype=float32)

In [ ]:
test_df['sub_pred'] = np.argmax(test_probs[0], axis = 1)
test_df['tox_pred'] = np.argmax(test_probs[1], axis = 1)
test_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,comment_text,subjectivity,toxicity,sub_pred,tox_pred
3,0001b41b1c6bb37e,more i can t make any real suggestions on impr...,0,0,0,0
14,00070ef96486d6f9,oh and the girl above started her arguments wi...,1,0,1,0
38,00169857adbc989b,hi explicit can you block o fenian for edit wa...,1,0,1,0
41,00173958f46763a2,tfd i think we just eced responded to each oth...,0,0,0,0
83,00349c6325526c11,april thank you for experimenting with the pag...,0,0,0,0


In [ ]:
from sklearn.metrics import f1_score,precision_score,recall_score

y_true = test_df['subjectivity'].values
y_pred = test_df['sub_pred'].values

print("subjectivity f1 score: " + str(f1_score(y_true, y_pred, average='micro')))
print("subjectivity precision score: " + str(precision_score(y_true, y_pred, average='micro')))
print("subjectivity recall score: " + str(recall_score(y_true, y_pred, average='micro')))
print('\n')
y_true = test_df['toxicity'].values
y_pred = test_df['tox_pred'].values
print("toxicity f1 score: " + str(f1_score(y_true, y_pred, average='micro')))
print("toxicity precision score: " + str(precision_score(y_true, y_pred, average='micro')))
print("toxicity recall score: " + str(recall_score(y_true, y_pred, average='micro')))

subjectivity f1 score: 0.8964155909261812
subjectivity precision score: 0.8964155909261812
subjectivity recall score: 0.8964155909261812


toxicity f1 score: 0.9481137987216444
toxicity precision score: 0.9481137987216444
toxicity recall score: 0.9481137987216444


In [ ]:
from collections import Counter

y_true = test_df['sub_pred'].values

Counter(y_true)

Counter({0: 10735, 1: 5223})

In [ ]:
y_true = test_df['tox_pred'].values

Counter(y_true)

Counter({0: 14313, 1: 1317, 2: 328})

## Load Self-labeled tweets Dataset From Text Corpus

In [ ]:
data_df = pd.read_csv(f'{data_folder}/preprocessed_neg_train.csv')
data_df.drop('id', axis=1, inplace=True)
data_df['comment_text'] = data_df['comment_text'].astype(str)
data_df.rename({'toxic':'toxicity'},axis=1,inplace=True)
data_df.dropna(inplace=True)
data_df.head()

,comment_text,subjectivity,toxicity
0,asaucegxd dont flex whore,1,1.0
1,lindseygrahamsc americans are ashamed of you h...,1,1.0
2,rt arvlnder came up off doge and bought a whol...,1,1.0
3,troche you gotta go to war and die smh,0,0.0
4,rt strongblacklead every legend has an origin ...,0,0.0


In [ ]:
test_df = pd.read_csv(f'{data_folder}/preprocessed_neg_test.csv')
test_df.head()

,comment_text,subjectivity,toxicity
0,rt trumpwarroom rt to wish a happy presidents ...,0,0
1,rt folkloredean the we needed therapy but inst...,1,0
2,rt rachelelisep i love that no one in texas ow...,1,0
3,vwow we have lady flotus45 our s shine brightr...,0,0
4,rt officialmxjp im first solo digital mini alb...,0,0


In [ ]:
y_true = data_df['subjectivity'].values

Counter(y_true)

Counter({0: 99424, 1: 44189})

In [ ]:
y_true = data_df['toxicity'].values

Counter(y_true)

Counter({0: 129848, 1: 12329, 2: 1436})

## Train on Manually Labelled Dataset

### Ensemble Model

In [ ]:
MODEL_NAMES = ['bert-base-multilingual-uncased','jplu/tf-xlm-roberta-base']

with strategy.scope():
    
    for i in range(len(MODEL_NAMES)):
      
      tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAMES[i])
      reviews = regular_encode(data_df.comment_text.astype(str).values.tolist(), tokenizer, maxlen=MAX_LEN)
  
      sss = StratifiedShuffleSplit(n_splits=4, random_state=42, test_size=0.2)
      X, y = reviews, data_df.subjectivity.astype(int).values
      z = data_df.toxicity.astype(int).values

      # loss = 'categorical_crossentropy'
      # models[i].compile(Adam(lr=1e-5), loss=[loss, loss],
      # metrics={'subjectivity':'accuracy', 'toxicity':'accuracy'})

      models[i].summary()

      for train_index, valid_index in sss.split(X, y):

          X_train, X_valid = X[train_index], X[valid_index]
          y_train, y_valid = y[train_index], y[valid_index]
          z_train, z_valid = z[train_index], z[valid_index]

          train_df = pd.DataFrame()
          train_df['sub'] = y_train
          train_df['tox'] = z_train
          label_cols = train_df.columns[:]
          labels = train_df[label_cols].values

          y_true = train_df['tox'].values
          print(Counter(y_true))
          y_true = train_df['sub'].values
          print(Counter(y_true))

          train_dataset = (
              tf.data.Dataset
              .from_tensor_slices((X_train,labels))
              .map(one_hot, num_parallel_calls=AUTO)
              .repeat()
              .shuffle(2048)
              .batch(BATCH_SIZE)
              .prefetch(AUTO)
          )

          valid_df = pd.DataFrame()
          valid_df['sub'] = y_valid
          valid_df['tox'] = z_valid
          label_cols = valid_df.columns[:]
          labels = valid_df[label_cols].values

          valid_dataset = (
              tf.data.Dataset
              .from_tensor_slices((X_valid, labels))
              .map(one_hot,num_parallel_calls=AUTO)
              .batch(BATCH_SIZE)
              .cache()
              .prefetch(AUTO)
          )

          n_steps = X_train.shape[0] // BATCH_SIZE
          models[i].fit(
              train_dataset,
              steps_per_epoch=n_steps,
              validation_data=valid_dataset,
              epochs=EPOCHS
          )
          models[i].save(f"{save_model_folder}/ensemble_model_{str(i)}.h5")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 192)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 167356416   input_word_ids[0][0]             
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 768)          0           bert[0][0]                       
__________________________________________________________________________________________________
dense (Dense)                   (None, 512)          393728      tf.__operators__.getitem[0][0]   
______________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 192)]        0                                            
__________________________________________________________________________________________________
roberta (TFRobertaMainLayer)    TFBaseModelOutputWit 278043648   input_word_ids[0][0]             
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 768)          0           roberta[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 512)          393728      tf.__operators__.getitem_1[0][0] 
____________________________________________________________________________________________

### Multi-head Model

In [ ]:
MODEL_NAMES = ['bert-base-multilingual-uncased','jplu/tf-xlm-roberta-base']

with strategy.scope():
        
    tokenizer1 = transformers.AutoTokenizer.from_pretrained(MODEL_NAMES[0])
    tokenizer2 = transformers.AutoTokenizer.from_pretrained(MODEL_NAMES[1])
    reviews1 = regular_encode(data_df.comment_text.astype(str).values.tolist(), tokenizer1, maxlen=MAX_LEN)
    reviews2 = regular_encode(data_df.comment_text.astype(str).values.tolist(), tokenizer2, maxlen=MAX_LEN)

    sss = StratifiedShuffleSplit(n_splits=4, random_state=42, test_size=0.2)
    X1, y1 = reviews1, data_df.subjectivity.astype(int).values
    z1 = data_df.toxicity.astype(int).values

    X2, y2 = reviews2, data_df.subjectivity.astype(int).values
    z2 = data_df.toxicity.astype(int).values

    model.summary()

    for train_index, valid_index in sss.split(X1, y1):

        X_train1, X_valid1 = X1[train_index], X1[valid_index]
        y_train1, y_valid1 = y1[train_index], y1[valid_index]
        z_train1, z_valid1 = z1[train_index], z1[valid_index]

        X_train2, X_valid2 = X2[train_index], X2[valid_index]
        y_train2, y_valid2 = y2[train_index], y2[valid_index]
        z_train2, z_valid2 = z2[train_index], z2[valid_index]

        train_df = pd.DataFrame()
        train_df['sub'] = y_train1
        train_df['tox'] = z_train1
        label_cols = train_df.columns[:]
        labels = train_df[label_cols].values
        data = np.array([X_train1,X_train2]).reshape(-1,2,192)

        print(data.shape)

        y_true = train_df['tox'].values
        print(Counter(y_true))
        y_true = train_df['sub'].values
        print(Counter(y_true))

        train_dataset = (
            tf.data.Dataset
            .from_tensor_slices(({"input_word_ids_1":X_train1,"input_word_ids_2":X_train2},labels))
            .map(one_hot_multi, num_parallel_calls=AUTO)
            .repeat()
            .shuffle(2048)
            .batch(BATCH_SIZE)
            .prefetch(AUTO)
        )

        valid_df = pd.DataFrame()
        valid_df['sub'] = y_valid1
        valid_df['tox'] = z_valid1
        label_cols = valid_df.columns[:]
        labels = valid_df[label_cols].values
        data = np.array([X_valid1,X_valid2]).reshape(-1,2,192)


        valid_dataset = (
            tf.data.Dataset
            .from_tensor_slices(({"input_word_ids_1":X_valid1,"input_word_ids_2":X_valid2},labels))
            .map(one_hot_multi, num_parallel_calls=AUTO)
            .batch(BATCH_SIZE)
            .cache()
            .prefetch(AUTO)
        )

        n_steps = X_train.shape[0] // BATCH_SIZE
        model.fit(
            train_dataset,
            steps_per_epoch=n_steps,
            validation_data=valid_dataset,
            epochs=EPOCHS
        )
        model.save(f"{save_model_folder}/multihead_model.h5")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids_1 (InputLayer)   [(None, 192)]        0                                            
__________________________________________________________________________________________________
input_word_ids_2 (InputLayer)   [(None, 192)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 167356416   input_word_ids_1[0][0]           
__________________________________________________________________________________________________
roberta (TFRobertaMainLayer)    TFBaseModelOutputWit 278043648   input_word_ids_2[0][0]           
______________________________________________________________________________________________

## Test Using Ensemble Model

### Load Trained Model

In [ ]:
df_sub = pd.read_csv(f'{data_folder}/preprocessed_kaggle_data.csv')

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Split Kaggle Dataset to Training set as well as test set to be used to test model after training
X_train, X_test, y_train, y_test = train_test_split(df_sub.comment_text.values, df_sub.id.values, test_size=0.10, random_state=42, stratify=df_sub[['toxic_class','neutral']])

data_df = df_sub[df_sub['id'].isin(y_train)]
data_df.rename({'neutral':'subjectivity','toxic_class':'toxicity'},axis=1,inplace=True)
data_df.drop('id', axis=1, inplace=True)
data_df['comment_text'] = data_df['comment_text'].astype(str)
test_df = pd.read_csv(f'{data_folder}/preprocessed_neg_test.csv')
test_df.head()
MAX_LEN = 192
EPOCHS = 4
BATCH_SIZE = 16 * strategy.num_replicas_in_sync # for ensemble model

def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
             texts, 
             truncation=True, 
             return_token_type_ids=False,
             pad_to_max_length=True,
             max_length=maxlen)
    
    return np.array(enc_di['input_ids'])

def build_model(transformer, model_name, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    if model_name == 'bert-base-multilingual-uncased':
      sequence_output = transformer.bert(input_word_ids)[0]
    elif model_name == 'jplu/tf-xlm-roberta-base':
      sequence_output = transformer.roberta(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    # x = Dense(1024, activation='relu')(cls_token)
    # x = Dropout(0.5)(x)
    x1 = Dense(512, activation='relu')(cls_token)
    x1 = Dropout(0.2)(x1)
    x1 = Dense(256, activation='relu')(x1)
    x1 = Dropout(0.2)(x1)
    x1 = Dense(128, activation='relu')(x1)
    x1 = Dropout(0.2)(x1)
    x2 = Dense(512, activation='relu')(cls_token)
    x2 = Dropout(0.2)(x2)
    x2 = Dense(256, activation='relu')(x2)
    x2 = Dropout(0.2)(x2)
    x2 = Dense(128, activation='relu')(x2)
    x2 = Dropout(0.2)(x2)
    sub = Dense(2, activation='softmax', name='subjectivity')(x1)
    toxic = Dense(3, activation='softmax', name='toxicity')(x2)
    
    model = Model(inputs=input_word_ids, outputs=[sub,toxic])
    loss = 'categorical_crossentropy'
    model.compile(Adam(lr=1e-5), loss=[loss, loss],
    metrics={'subjectivity':'accuracy', 'toxicity':'accuracy'})
    
    return model

MODEL_NAMES = ['bert-base-multilingual-uncased','jplu/tf-xlm-roberta-base']
models = [0 for i in range(len(MODEL_NAMES))]
test_probs = [0 for i in range(len(MODEL_NAMES))]
test_results = [0 for i in range(len(MODEL_NAMES))]

with strategy.scope():
    
    for i in range(len(MODEL_NAMES)):
        
        models[i] = transformers.TFAutoModel.from_pretrained(MODEL_NAMES[i])
        # models[i].trainable = False
        
        tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAMES[i])
        reviews = regular_encode(data_df.comment_text.astype(str).values.tolist(), tokenizer, maxlen=MAX_LEN)
    
        sss = StratifiedShuffleSplit(n_splits=4, random_state=42, test_size=0.2)
        X, y = reviews, data_df.subjectivity.astype(int).values
        z = data_df.toxicity.astype(int).values

        models[i] = build_model(models[i], MODEL_NAMES[i], max_len=MAX_LEN)
        models[i].summary()

In [ ]:
MODEL_NAMES = ['bert-base-multilingual-uncased','jplu/tf-xlm-roberta-base']
models = [0 for i in range(len(MODEL_NAMES))]
input_word_ids = Input(shape=(192,), dtype=tf.int32, name="input_word_ids")
# bert = transformers.TFAutoModel.from_pretrained(MODEL_NAMES[0]).bert(input_word_ids)[0]
# roberta = transformers.TFAutoModel.from_pretrained(MODEL_NAMES[1]).roberta(input_word_ids)[0]

with strategy.scope():

  models[0] = tf.keras.models.load_model(f'{save_model_folder}/ensemble_model_0.h5')
  models[1] = tf.keras.models.load_model(f'{save_model_folder}/ensemble_model_1.h5')

In [ ]:
 models[0].summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 192)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 167356416   input_word_ids[0][0]             
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 768)          0           bert[0][0]                       
__________________________________________________________________________________________________
dense (Dense)                   (None, 512)          393728      tf.__operators__.getitem[0][0]   
______________________________________________________________________________________________

### Predict on Trained Ensemble Model

In [ ]:
MODEL_NAMES = ['bert-base-multilingual-uncased','jplu/tf-xlm-roberta-base']
test_probs = [0 for i in range(len(MODEL_NAMES))]

with strategy.scope():
    
    for i in range(len(MODEL_NAMES)):
      print(MODEL_NAMES[i])
      tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAMES[i])
      test_reviews = regular_encode(test_df.comment_text.astype(str).values.tolist(), tokenizer, maxlen=192)

      test_dataset = (
      tf.data.Dataset
      .from_tensor_slices((test_reviews))
      .batch(128)
      .prefetch(AUTO)
      )

      test_probs[i] = models[i].predict(test_dataset, verbose=1)

bert-base-multilingual-uncased


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


14/14 [==============================] - 14s 746ms/step
jplu/tf-xlm-roberta-base
14/14 [==============================] - 15s 842ms/step


## Averaging Ensemble Model Results

In [ ]:
test_df['sub_pred'] = np.argmax(((test_probs[0][0] + test_probs[1][0])/2), axis = 1)
test_df['tox_pred'] = np.argmax(((test_probs[0][1] + test_probs[1][1])/2), axis = 1)
test_df.head()

,comment_text,subjectivity,toxicity,sub_pred,tox_pred
0,rt trumpwarroom rt to wish a happy presidents ...,0,0,0,0
1,rt folkloredean the we needed therapy but inst...,1,0,0,0
2,rt rachelelisep i love that no one in texas ow...,1,0,0,0
3,vwow we have lady flotus45 our s shine brightr...,0,0,1,0
4,rt officialmxjp im first solo digital mini alb...,0,0,0,0


In [ ]:
from sklearn.metrics import f1_score,precision_score,recall_score

y_true = test_df['subjectivity'].values
y_pred = test_df['sub_pred'].values

print("subjectivity f1 score: " + str(f1_score(y_true, y_pred, average='micro')))
print("subjectivity precision score: " + str(precision_score(y_true, y_pred, average='micro')))
print("subjectivity recall score: " + str(recall_score(y_true, y_pred, average='micro')))
print('\n')
y_true = test_df['toxicity'].values
y_pred = test_df['tox_pred'].values
print("toxicity f1 score: " + str(f1_score(y_true, y_pred, average='micro')))
print("toxicity precision score: " + str(precision_score(y_true, y_pred, average='micro')))
print("toxicity recall score: " + str(recall_score(y_true, y_pred, average='micro')))

subjectivity f1 score: 0.8448769318832284
subjectivity precision score: 0.8448769318832284
subjectivity recall score: 0.8448769318832284


toxicity f1 score: 0.8775042930738409
toxicity precision score: 0.8775042930738409
toxicity recall score: 0.8775042930738409


In [ ]:
y_true = test_df['subjectivity'].values
y_pred = test_df['sub_pred'].values

print("subjectivity f1 score: " + str(f1_score(y_true, y_pred, average=None)))
print("subjectivity precision score: " + str(precision_score(y_true, y_pred, average=None)))
print("subjectivity recall score: " + str(recall_score(y_true, y_pred, average=None)))
print('\n')
y_true = test_df['toxicity'].values
y_pred = test_df['tox_pred'].values
print("toxicity f1 score: " + str(f1_score(y_true, y_pred, average=None)))
print("toxicity precision score: " + str(precision_score(y_true, y_pred, average=None)))
print("toxicity recall score: " + str(recall_score(y_true, y_pred, average=None)))

subjectivity f1 score: [0.87585891 0.79328757]
subjectivity precision score: [0.88110599 0.78549849]
subjectivity recall score: [0.87067395 0.80123267]


toxicity f1 score: [0.93964583 0.60903733 0.71559633]
toxicity precision score: [0.93998554 0.63265306 0.65546218]
toxicity recall score: [0.93930636 0.58712121 0.78787879]


In [ ]:
from collections import Counter

y_true = test_df['sub_pred'].values

Counter(y_true)

Counter({0: 1085, 1: 662})

In [ ]:
y_true = test_df['subjectivity'].values

Counter(y_true)

Counter({0: 1098, 1: 649})

In [ ]:
y_true = test_df['tox_pred'].values

Counter(y_true)

Counter({0: 1369, 1: 248, 2: 130})

In [ ]:
y_true = test_df['toxicity'].values

Counter(y_true)

Counter({0: 1384, 1: 264, 2: 99})

## Test on Multi-head Model

In [ ]:
test_df = pd.read_csv(f'{data_folder}/preprocessed_neg_test.csv')
test_df.head()
BATCH_SIZE = 8 * strategy.num_replicas_in_sync  # for multihead model
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
             texts, 
             truncation=True, 
             return_token_type_ids=False,
             pad_to_max_length=True,
             max_length=maxlen)
    
    return np.array(enc_di['input_ids'])

def build_model(max_len=512):
    
    transformer = transformers.TFAutoModel.from_pretrained('bert-base-multilingual-uncased')
    transformer2 = transformers.TFAutoModel.from_pretrained('jplu/tf-xlm-roberta-base')
    input_word_ids_1 = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids_1")
    input_word_ids_2 = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids_2")
    sequence_output1 = transformer.bert(input_word_ids_1)[0]
    sequence_output2 = transformer2.roberta(input_word_ids_2)[0]
    cls_token1 = sequence_output1[:, 0, :]
    cls_token2 = sequence_output2[:, 0, :]
    x = Average()([cls_token1,cls_token2])
    # x = Dropout(0.5)(x)
    x1 = Dense(512, activation='relu')(x)
    x1 = Dropout(0.2)(x1)
    x1 = Dense(256, activation='relu')(x1)
    x1 = Dropout(0.2)(x1)
    x1 = Dense(128, activation='relu')(x1)
    x1 = Dropout(0.2)(x1)
    x2 = Dense(512, activation='relu')(x)
    x2 = Dropout(0.2)(x2)
    x2 = Dense(256, activation='relu')(x2)
    x2 = Dropout(0.2)(x2)
    x2 = Dense(128, activation='relu')(x2)
    x2 = Dropout(0.2)(x2)
    sub = Dense(2, activation='softmax', name='subjectivity')(x1)
    toxic = Dense(3, activation='softmax', name='toxicity')(x2)
    
    model = Model(inputs=[input_word_ids_1,input_word_ids_2], outputs=[sub,toxic])
    loss = tfa.losses.SigmoidFocalCrossEntropy(reduction=tf.keras.losses.Reduction.AUTO)
    model.compile(Adam(lr=1e-6), loss=[loss, loss],
    metrics={'subjectivity':'accuracy', 'toxicity':'accuracy'})
    
    return model

MODEL_NAMES = ['bert-base-multilingual-uncased','jplu/tf-xlm-roberta-base']
models = [0 for i in range(len(MODEL_NAMES))]
test_probs = [0 for i in range(len(MODEL_NAMES))]
test_results = [0 for i in range(len(MODEL_NAMES))]

with strategy.scope():
        
    tokenizer1 = transformers.AutoTokenizer.from_pretrained(MODEL_NAMES[0])
    tokenizer2 = transformers.AutoTokenizer.from_pretrained(MODEL_NAMES[1])
    reviews1 = regular_encode(data_df.comment_text.astype(str).values.tolist(), tokenizer1, maxlen=MAX_LEN)
    reviews2 = regular_encode(data_df.comment_text.astype(str).values.tolist(), tokenizer2, maxlen=MAX_LEN)

    sss = StratifiedShuffleSplit(n_splits=4, random_state=42, test_size=0.2)
    X1, y1 = reviews1, data_df.subjectivity.astype(int).values
    z1 = data_df.toxicity.astype(int).values

    X2, y2 = reviews2, data_df.subjectivity.astype(int).values
    z2 = data_df.toxicity.astype(int).values

    model = build_model(max_len=MAX_LEN)
    model.summary()

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Some layers from the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Some layers from the model checkpoint at jplu/tf-xlm-roberta-base were not used when initializing TFXLMRobertaModel: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaModel were initialized from the model checkpoint at jplu/tf-xlm-roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids_1 (InputLayer)   [(None, 192)]        0                                            
__________________________________________________________________________________________________
input_word_ids_2 (InputLayer)   [(None, 192)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 167356416   input_word_ids_1[0][0]           
__________________________________________________________________________________________________
roberta (TFRobertaMainLayer)    TFBaseModelOutputWit 278043648   input_word_ids_2[0][0]           
______________________________________________________________________________________________

In [ ]:
test_df = pd.read_csv(f'{data_folder}/preprocessed_neg_test.csv')
test_df.head()
MODEL_NAMES = ['bert-base-multilingual-uncased','jplu/tf-xlm-roberta-base']
# model = tf.keras.models.load_model(f'{save_model_folder}/multihead_model.h5')

with strategy.scope():
    
    tokenizer1 = transformers.AutoTokenizer.from_pretrained(MODEL_NAMES[0])
    tokenizer2 = transformers.AutoTokenizer.from_pretrained(MODEL_NAMES[1])
    test_reviews1 = regular_encode(test_df.comment_text.astype(str).values.tolist(), tokenizer1, maxlen=MAX_LEN)
    test_reviews2 = regular_encode(test_df.comment_text.astype(str).values.tolist(), tokenizer2, maxlen=MAX_LEN)

    test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(({"input_word_ids_1":test_reviews1,"input_word_ids_2":test_reviews2}))
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
    )

    test_probs = model.predict(test_dataset, verbose=1)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


28/28 [==============================] - 2s 59ms/step


## Multi-head Model Results

In [ ]:
test_df['sub_pred'] = np.argmax(test_probs[0], axis = 1)
test_df['tox_pred'] = np.argmax(test_probs[1], axis = 1)
test_df.head()

,comment_text,subjectivity,toxicity,sub_pred,tox_pred
0,rt trumpwarroom rt to wish a happy presidents ...,0,0,0,0
1,rt folkloredean the we needed therapy but inst...,1,0,0,0
2,rt rachelelisep i love that no one in texas ow...,1,0,1,0
3,vwow we have lady flotus45 our s shine brightr...,0,0,1,1
4,rt officialmxjp im first solo digital mini alb...,0,0,0,0


In [ ]:
from sklearn.metrics import f1_score,precision_score,recall_score

y_true = test_df['subjectivity'].values
y_pred = test_df['sub_pred'].values

print("subjectivity f1 score: " + str(f1_score(y_true, y_pred, average='micro')))
print("subjectivity precision score: " + str(precision_score(y_true, y_pred, average='micro')))
print("subjectivity recall score: " + str(recall_score(y_true, y_pred, average='micro')))
print('\n')
y_true = test_df['toxicity'].values
y_pred = test_df['tox_pred'].values
print("toxicity f1 score: " + str(f1_score(y_true, y_pred, average='micro')))
print("toxicity precision score: " + str(precision_score(y_true, y_pred, average='micro')))
print("toxicity recall score: " + str(recall_score(y_true, y_pred, average='micro')))

subjectivity f1 score: 0.8374356038923868
subjectivity precision score: 0.8374356038923869
subjectivity recall score: 0.8374356038923869


toxicity f1 score: 0.8534630795649685
toxicity precision score: 0.8534630795649685
toxicity recall score: 0.8534630795649685


In [ ]:
y_true = test_df['subjectivity'].values
y_pred = test_df['sub_pred'].values

print("subjectivity f1 score: " + str(f1_score(y_true, y_pred, average=None)))
print("subjectivity precision score: " + str(precision_score(y_true, y_pred, average=None)))
print("subjectivity recall score: " + str(recall_score(y_true, y_pred, average=None)))
print('\n')
y_true = test_df['toxicity'].values
y_pred = test_df['tox_pred'].values
print("toxicity f1 score: " + str(f1_score(y_true, y_pred, average=None)))
print("toxicity precision score: " + str(precision_score(y_true, y_pred, average=None)))
print("toxicity recall score: " + str(recall_score(y_true, y_pred, average=None)))

subjectivity f1 score: [0.87031963 0.78220859]
subjectivity precision score: [0.87271062 0.77862595]
subjectivity recall score: [0.86794171 0.78582435]


toxicity f1 score: [0.93016453 0.55952381 0.61176471]
toxicity precision score: [0.9415248 0.5875    0.5      ]
toxicity recall score: [0.91907514 0.53409091 0.78787879]


In [ ]:
from collections import Counter

y_true = test_df['sub_pred'].values

Counter(y_true)

Counter({0: 1092, 1: 655})

In [ ]:
y_true = test_df['subjectivity'].values

Counter(y_true)

Counter({0: 1098, 1: 649})

In [ ]:
y_true = test_df['tox_pred'].values

Counter(y_true)

Counter({0: 1351, 1: 240, 2: 156})

In [ ]:
y_true = test_df['toxicity'].values

Counter(y_true)

Counter({0: 1384, 1: 264, 2: 99})

## Label Unlabelled Crawled Data Using Trained Model

In [ ]:
df = pd.read_csv(f'{data_folder}/toxic_tweets_combined_url.csv')
df.head()

,tweet_id,text,created_date_time,username,user_screen_name,user_id,user_location,user_description,verified,associated_place,retweet_count,location,user_geo,url
0,1361169823707918336,It was Dio all along! Happy dumb holiday! 🖤 ht...,2021-02-15 04:25:49,𝕕𝕒𝕟𝕚𝕖𝕝𝕝𝕖 𝕤𝕝𝕒𝕦𝕥𝕖𝕣,DanielleSlauter,66803433,"Indianapolis, Marion, Indiana, United States",•✞•Level 26•INFJ•Aries•anime trash•@waifuwatch...,False,NaN,0,"Indianapolis, Marion, Indiana, United States","(39.7683331, -86.1583502, 0.0)",https://twitter.com/twitter/status/13611698237...
1,1360681569527222275,RT @AndrewStoeten: We could stop the pandemic....,2021-02-13 20:05:41,jordan meuIendyk,jordym1nine,19632321,"Toronto, Golden Horseshoe, Ontario, Canada",former child actor. chief of the BPD. human sh...,False,NaN,166,"Toronto, Golden Horseshoe, Ontario, Canada","(43.6534817, -79.3839347, 0.0)",https://twitter.com/twitter/status/13606815695...
2,1359821276269993985,RT @TachaSmar: @Blessin50078066 @Chinyere2231 ...,2021-02-11 11:07:11,Renaissance🔱 ˢᵀ,renegadelia,1228398056602787841,"Universal, Veranópolis, Região Geográfica Imed...","Write your own verse!\r\n\r\nI am abundance, I...",False,NaN,7,"Universal, Veranópolis, Região Geográfica Imed...","(-28.9182121, -51.547104, 0.0)",https://twitter.com/twitter/status/13598212762...
3,1360730455146975238,not me going off on a grown man in the middle ...,2021-02-13 23:19:56,paula,uhhpaula,4540327527,"South Florida, Columbus, Cherokee County, Kans...",aries,False,NaN,0,"South Florida, Columbus, Cherokee County, Kans...","(37.1677128, -94.8464256, 0.0)",https://twitter.com/twitter/status/13607304551...
4,1360393239107403778,RT @SenDuckworth: If you go just 10 stops down...,2021-02-13 00:59:57,Shannon Fx Halbur,UnderTallSWH,3212694291,"Raleigh, Wake County, North Carolina, United S...","Gun Violence Prevention Activist, with Moms De...",False,NaN,276,"Raleigh, Wake County, North Carolina, United S...","(35.7803977, -78.6390989, 0.0)",https://twitter.com/twitter/status/13603932391...


In [ ]:
def emoji_cleaning(text):
    
  # Change emoji to text
  text = emoji.demojize(text).replace(":", " ")
  
  # Delete repeated emoji
  tokenizer = text.split()
  repeated_list = []
  
  for word in tokenizer:
      if word not in repeated_list:
          repeated_list.append(word)
  
  text = ' '.join(text for text in repeated_list)
  text = text.replace("_", " ").replace("-", " ")
  return text

def clean_smileys(text):
    
    text = re.sub(r'(:\)|: \)|\(\:|:-\)|: -\)|: - \)|:D|: D)', ' smile ', text)
    text = re.sub(r'(:\(|: \(|\)\:|:-\(|: -\(|: - \(|:\'\()', ' dislike ', text)
    text = re.sub(r'(<3)', ' heart ', text)
    text = re.sub(r'(:/)', ' dislike ', text)
    text = re.sub(r'(;\)|; \))', ' wink ', text)
    return ' '.join([word for word in text.split()])

def decontracted(text):
  text = re.sub(r"won\'t", "will not", text)
  text = re.sub(r"can\'t", "can not", text)
  text = re.sub(r"n\'t", " not", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'s", " is", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'t", " not", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'m", " am", text)

  text = re.sub(r"n\'t", " not", text)
  text = re.sub(r"n\’t", " not", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\’re", " are", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\’d", " would", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\’ll", " will", text)
  text = re.sub(r"\'t", " not", text)
  text = re.sub(r"\’t", " not", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\’ve", " have", text)
  text = re.sub(r"\'m", " am", text)
  text = re.sub(r"\’m", " am", text)
  text = re.sub(r"\“", "", text)
  text = re.sub(r"\”", "", text)
  text = re.sub(r"\…", "", text)

  return text

def clean_text(text):
  text = str(text)
  text = re.sub(r'(\w)\1{2,}', r'\1', text)
  text = re.sub(r'[^a-zA-Z ]+', ' ', text)
  text = re.sub(r'http\S+', ' ', text)
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
  text = re.sub(r'^RT[\s]+', '', text)
  text = re.sub(r'pic.twitter\S+', ' ', text)
  text = re.sub(r'#', '', text)
  text = text.lower()

  return text

In [ ]:
ul_df = df[["tweet_id","text"]]
ul_df.head()

,tweet_id,text
0,1361169823707918336,It was Dio all along! Happy dumb holiday! 🖤 ht...
1,1360681569527222275,RT @AndrewStoeten: We could stop the pandemic....
2,1359821276269993985,RT @TachaSmar: @Blessin50078066 @Chinyere2231 ...
3,1360730455146975238,not me going off on a grown man in the middle ...
4,1360393239107403778,RT @SenDuckworth: If you go just 10 stops down...


In [ ]:
ul_df['text'] = ul_df['text'].apply(clean_text).apply(emoji_cleaning).apply(clean_smileys).apply(decontracted)
ul_df.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,tweet_id,text
0,1361169823707918336,it was dio all along happy dumb holiday t co l...
1,1360681569527222275,andrewstoeten we could stop the pandemic our l...
2,1359821276269993985,tachasmar blessin chinyere in other words you ...
3,1360730455146975238,not me going off on a grown man in the middle ...
4,1360393239107403778,senduckworth if you go just stops down the sub...
5,1359944829845663744,you should tell that to the pt sexually assaul...
6,1359937609435340811,gutterbumpkin and how to die not while executi...
7,1361171338874994690,realcandaceo since when do evangelical leaders...
8,1359988095316185088,tishertisher it s the darkest shit imaginable ...
9,1359971602658701315,i was told a supervisor would go over the call...


In [ ]:
MODEL_NAMES = ['bert-base-multilingual-uncased','jplu/tf-xlm-roberta-base']
test_probs = [0 for i in range(len(MODEL_NAMES))]

with strategy.scope():
    
    for i in range(len(MODEL_NAMES)):
      print(MODEL_NAMES[i])
      tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAMES[i])
      test_reviews = regular_encode(ul_df.text.astype(str).values.tolist(), tokenizer, maxlen=MAX_LEN)

      test_dataset = (
      tf.data.Dataset
      .from_tensor_slices((test_reviews))
      .batch(BATCH_SIZE)
      .prefetch(AUTO)
      )

      test_probs[i] = models[i].predict(test_dataset, verbose=1)

bert-base-multilingual-uncased


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


31/31 [==============================] - 5s 150ms/step
jplu/tf-xlm-roberta-base
31/31 [==============================] - 5s 155ms/step


In [ ]:
ul_df['subjectivity'] = np.argmax(((test_probs[0][0] + test_probs[1][0])/2), axis = 1)
ul_df['toxicity'] = np.argmax(((test_probs[0][1] + test_probs[1][1])/2), axis = 1)
ul_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,tweet_id,text,subjectivity,toxicity
0,1361169823707918336,it was dio all along happy dumb holiday t co l...,1,0
1,1360681569527222275,andrewstoeten we could stop the pandemic our l...,0,0
2,1359821276269993985,tachasmar blessin chinyere in other words you ...,1,1
3,1360730455146975238,not me going off on a grown man in the middle ...,0,0
4,1360393239107403778,senduckworth if you go just stops down the sub...,0,0


In [ ]:
df['subjectivity'] = np.argmax(((test_probs[0][0] + test_probs[1][0])/2), axis = 1)
df['toxicity'] = np.argmax(((test_probs[0][1] + test_probs[1][1])/2), axis = 1)
df.head()

,tweet_id,text,created_date_time,username,user_screen_name,user_id,user_location,user_description,verified,associated_place,retweet_count,location,user_geo,url,subjectivity,toxicity
0,1361169823707918336,It was Dio all along! Happy dumb holiday! 🖤 ht...,2021-02-15 04:25:49,𝕕𝕒𝕟𝕚𝕖𝕝𝕝𝕖 𝕤𝕝𝕒𝕦𝕥𝕖𝕣,DanielleSlauter,66803433,"Indianapolis, Marion, Indiana, United States",•✞•Level 26•INFJ•Aries•anime trash•@waifuwatch...,False,NaN,0,"Indianapolis, Marion, Indiana, United States","(39.7683331, -86.1583502, 0.0)",https://twitter.com/twitter/status/13611698237...,1,0
1,1360681569527222275,RT @AndrewStoeten: We could stop the pandemic....,2021-02-13 20:05:41,jordan meuIendyk,jordym1nine,19632321,"Toronto, Golden Horseshoe, Ontario, Canada",former child actor. chief of the BPD. human sh...,False,NaN,166,"Toronto, Golden Horseshoe, Ontario, Canada","(43.6534817, -79.3839347, 0.0)",https://twitter.com/twitter/status/13606815695...,0,0
2,1359821276269993985,RT @TachaSmar: @Blessin50078066 @Chinyere2231 ...,2021-02-11 11:07:11,Renaissance🔱 ˢᵀ,renegadelia,1228398056602787841,"Universal, Veranópolis, Região Geográfica Imed...","Write your own verse!\r\n\r\nI am abundance, I...",False,NaN,7,"Universal, Veranópolis, Região Geográfica Imed...","(-28.9182121, -51.547104, 0.0)",https://twitter.com/twitter/status/13598212762...,1,1
3,1360730455146975238,not me going off on a grown man in the middle ...,2021-02-13 23:19:56,paula,uhhpaula,4540327527,"South Florida, Columbus, Cherokee County, Kans...",aries,False,NaN,0,"South Florida, Columbus, Cherokee County, Kans...","(37.1677128, -94.8464256, 0.0)",https://twitter.com/twitter/status/13607304551...,0,0
4,1360393239107403778,RT @SenDuckworth: If you go just 10 stops down...,2021-02-13 00:59:57,Shannon Fx Halbur,UnderTallSWH,3212694291,"Raleigh, Wake County, North Carolina, United S...","Gun Violence Prevention Activist, with Moms De...",False,NaN,276,"Raleigh, Wake County, North Carolina, United S...","(35.7803977, -78.6390989, 0.0)",https://twitter.com/twitter/status/13603932391...,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3846 entries, 0 to 3845
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   tweet_id           3846 non-null   int64 
 1   text               3846 non-null   object
 2   created_date_time  3846 non-null   object
 3   username           3846 non-null   object
 4   user_screen_name   3846 non-null   object
 5   user_id            3846 non-null   int64 
 6   user_location      3846 non-null   object
 7   user_description   3544 non-null   object
 8   verified           3846 non-null   bool  
 9   associated_place   60 non-null     object
 10  retweet_count      3846 non-null   int64 
 11  location           3846 non-null   object
 12  user_geo           3846 non-null   object
 13  url                3846 non-null   object
 14  subjectivity       3846 non-null   int64 
 15  toxicity           3846 non-null   int64 
dtypes: bool(1), int64(5), object(10)
memory us

In [ ]:
df['tweet_id'] = df['tweet_id'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3846 entries, 0 to 3845
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   tweet_id           3846 non-null   object
 1   text               3846 non-null   object
 2   created_date_time  3846 non-null   object
 3   username           3846 non-null   object
 4   user_screen_name   3846 non-null   object
 5   user_id            3846 non-null   int64 
 6   user_location      3846 non-null   object
 7   user_description   3544 non-null   object
 8   verified           3846 non-null   bool  
 9   associated_place   60 non-null     object
 10  retweet_count      3846 non-null   int64 
 11  location           3846 non-null   object
 12  user_geo           3846 non-null   object
 13  url                3846 non-null   object
 14  subjectivity       3846 non-null   int64 
 15  toxicity           3846 non-null   int64 
dtypes: bool(1), int64(4), object(11)
memory us

In [ ]:
df.to_csv(f'{data_folder}/negative_tweets_labeled.csv',index=False)

In [ ]:
df = pd.read_csv(f'{data_folder}/unlabelled_data.csv')
df.head()

,text,created_date_time,tweet_id,username,user_screen_name,user_id,user_location,user_description,verified,associated_place,retweet_count,user_geo,link,url
0,RT @TrumpWarRoom___: RT to wish a Happy Presid...,2021-02-15 19:00:53,1361390039108907008,VaQuireboy,VaQuireboy,742076049618538496,"Virginia, United States","Family man, College educated, Independent, Vie...",False,NaN,2124,"(37.1232245, -78.4927721, 0.0)",https://twitter.com/twitter/status/13613900391...,NaN
1,"RT @Rules_twt: If ""uncomfortable"" was a pictur...",2021-02-15 19:00:48,1361390020008038402,𝓐𝓷𝓪 𝓒𝓻𝓲𝓼𝓽𝓲𝓷𝓪 ✨,bemAtoaAqui,1182740439184658438,"Moura, Beja, Baixo Alentejo, Alentejo, Portugal",♐️//#SimAosToiros// F 💓,False,NaN,613,"(38.145868899999996, -7.36681873826084, 0.0)",https://twitter.com/twitter/status/13613900200...,NaN
2,RT @_ixcato: I think he's neat. :) #criticalro...,2021-02-15 19:00:47,1361390012466667520,Spectre’s Icy Touch,spectrealafete,40359764,"Toronto, Golden Horseshoe, Ontario, Canada",Mostly just flailing about changelings... and ...,False,NaN,108,"(43.6534817, -79.3839347, 0.0)",https://twitter.com/twitter/status/13613900124...,NaN
3,RT @Tomas_Verde: Who's the brave soul who is g...,2021-02-15 19:00:56,1361390051448545280,Taylor Moats,idigmoats,836340324205199361,"Whitemarsh Island, Chatham County, Georgia, 31...",NaN,False,NaN,26,"(32.0327121, -81.0142786, 0.0)",https://twitter.com/twitter/status/13613900514...,NaN
4,@kenndold @RachelFields_ I really hope this is...,2021-02-15 19:00:53,1361390038056075267,Alejandra Felix,alexxfelixx95,1297156705,"Cambridge, Middlesex County, Massachusetts, Un...",tiny girl pretending to do chemistry by day @H...,False,NaN,0,"(42.3750997, -71.1056157, 0.0)",https://twitter.com/twitter/status/13613900380...,NaN


In [ ]:
ul_df = df[["tweet_id","text"]]
ul_df.head()

,tweet_id,text
0,1361390039108907008,RT @TrumpWarRoom___: RT to wish a Happy Presid...
1,1361390020008038402,"RT @Rules_twt: If ""uncomfortable"" was a pictur..."
2,1361390012466667520,RT @_ixcato: I think he's neat. :) #criticalro...
3,1361390051448545280,RT @Tomas_Verde: Who's the brave soul who is g...
4,1361390038056075267,@kenndold @RachelFields_ I really hope this is...


In [ ]:
ul_df['text'] = ul_df['text'].apply(clean_text).apply(emoji_cleaning).apply(clean_smileys).apply(decontracted)
ul_df.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,tweet_id,text
0,1361390039108907008,trumpwarroom rt to wish a happy president s da...
1,1361390020008038402,rules twt if uncomfortable was a picture t co ...
2,1361390012466667520,ixcato i think he s neat criticalrolefanart cr...
3,1361390051448545280,tomas verde who s the brave soul is going to a...
4,1361390038056075267,kenndold rachelfields i really hope this is th...
5,1361390000630157313,rwpusa solicitation of election fraud is a fel...
6,1361390010499354625,pwcdeez wait they charge if i was a guest d do...
7,1361390030288269314,veevyan when a nigga come into your life he co...
8,1361390047426134017,folkloredean the we needed therapy but instead...
9,1361389993097330690,gregabbott tx looks like defunding police didn...


In [ ]:
MODEL_NAMES = ['bert-base-multilingual-uncased','jplu/tf-xlm-roberta-base']
test_probs = [0 for i in range(len(MODEL_NAMES))]

with strategy.scope():
    
    for i in range(len(MODEL_NAMES)):
      print(MODEL_NAMES[i])
      tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAMES[i])
      test_reviews = regular_encode(ul_df.text.astype(str).values.tolist(), tokenizer, maxlen=MAX_LEN)

      test_dataset = (
      tf.data.Dataset
      .from_tensor_slices((test_reviews))
      .batch(BATCH_SIZE)
      .prefetch(AUTO)
      )

      test_probs[i] = models[i].predict(test_dataset, verbose=1)

bert-base-multilingual-uncased


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


86/86 [==============================] - 10s 104ms/step
jplu/tf-xlm-roberta-base
86/86 [==============================] - 9s 103ms/step


In [ ]:
df['subjectivity'] = np.argmax(((test_probs[0][0] + test_probs[1][0])/2), axis = 1)
df['toxicity'] = np.argmax(((test_probs[0][1] + test_probs[1][1])/2), axis = 1)
df.head()

,text,created_date_time,tweet_id,username,user_screen_name,user_id,user_location,user_description,verified,associated_place,retweet_count,user_geo,link,url,subjectivity,toxicity
0,RT @TrumpWarRoom___: RT to wish a Happy Presid...,2021-02-15 19:00:53,1361390039108907008,VaQuireboy,VaQuireboy,742076049618538496,"Virginia, United States","Family man, College educated, Independent, Vie...",False,NaN,2124,"(37.1232245, -78.4927721, 0.0)",https://twitter.com/twitter/status/13613900391...,NaN,1,0
1,"RT @Rules_twt: If ""uncomfortable"" was a pictur...",2021-02-15 19:00:48,1361390020008038402,𝓐𝓷𝓪 𝓒𝓻𝓲𝓼𝓽𝓲𝓷𝓪 ✨,bemAtoaAqui,1182740439184658438,"Moura, Beja, Baixo Alentejo, Alentejo, Portugal",♐️//#SimAosToiros// F 💓,False,NaN,613,"(38.145868899999996, -7.36681873826084, 0.0)",https://twitter.com/twitter/status/13613900200...,NaN,1,0
2,RT @_ixcato: I think he's neat. :) #criticalro...,2021-02-15 19:00:47,1361390012466667520,Spectre’s Icy Touch,spectrealafete,40359764,"Toronto, Golden Horseshoe, Ontario, Canada",Mostly just flailing about changelings... and ...,False,NaN,108,"(43.6534817, -79.3839347, 0.0)",https://twitter.com/twitter/status/13613900124...,NaN,1,0
3,RT @Tomas_Verde: Who's the brave soul who is g...,2021-02-15 19:00:56,1361390051448545280,Taylor Moats,idigmoats,836340324205199361,"Whitemarsh Island, Chatham County, Georgia, 31...",NaN,False,NaN,26,"(32.0327121, -81.0142786, 0.0)",https://twitter.com/twitter/status/13613900514...,NaN,1,0
4,@kenndold @RachelFields_ I really hope this is...,2021-02-15 19:00:53,1361390038056075267,Alejandra Felix,alexxfelixx95,1297156705,"Cambridge, Middlesex County, Massachusetts, Un...",tiny girl pretending to do chemistry by day @H...,False,NaN,0,"(42.3750997, -71.1056157, 0.0)",https://twitter.com/twitter/status/13613900380...,NaN,1,0


In [ ]:
df['tweet_id'] = df['tweet_id'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10993 entries, 0 to 10992
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               10993 non-null  object
 1   created_date_time  10993 non-null  object
 2   tweet_id           10993 non-null  object
 3   username           10992 non-null  object
 4   user_screen_name   10993 non-null  object
 5   user_id            10993 non-null  int64 
 6   user_location      10993 non-null  object
 7   user_description   10161 non-null  object
 8   verified           10993 non-null  bool  
 9   associated_place   179 non-null    object
 10  retweet_count      10993 non-null  int64 
 11  user_geo           10993 non-null  object
 12  link               5147 non-null   object
 13  url                5846 non-null   object
 14  subjectivity       10993 non-null  int64 
 15  toxicity           10993 non-null  int64 
dtypes: bool(1), int64(4), object(11)
memory 

In [ ]:
df.to_csv(f'{data_folder}/neutral_tweets_labeled.csv',index=False)